## Pregunta 2

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%%time 
!mkdir train_images
!mkdir test_images
zip_train_path = '/content/gdrive/My Drive/Colab Notebooks/train_images.zip'
zip_test_path = '/content/gdrive/My Drive/Colab Notebooks/test_images.zip'
!cp "{zip_train_path}" .
!cp "{zip_test_path}" .
!unzip -q train_images.zip -d train_images/
!unzip -q test_images.zip -d test_images/
!rm train_images.zip
!rm test_images.zip

mkdir: cannot create directory ‘train_images’: File exists
mkdir: cannot create directory ‘test_images’: File exists
cp: cannot stat '/content/gdrive/My Drive/Colab Notebooks/train_images.zip': Transport endpoint is not connected
cp: cannot stat '/content/gdrive/My Drive/Colab Notebooks/test_images.zip': Transport endpoint is not connected
unzip:  cannot find or open train_images.zip, train_images.zip.zip or train_images.zip.ZIP.
unzip:  cannot find or open test_images.zip, test_images.zip.zip or test_images.zip.ZIP.
rm: cannot remove 'train_images.zip': No such file or directory
rm: cannot remove 'test_images.zip': No such file or directory
CPU times: user 69 ms, sys: 39.5 ms, total: 108 ms
Wall time: 13.1 s


### Se importan los datos

Para la división de los datos se utilizo un 80% para datos de entrenamiento y un 20% para el conjunto de test, esto se hizo para elegir un modelo que tengo lo menor posible overfitting en el conjunto  de entrenamiento.

In [4]:
import numpy as np 
import pandas as pd
from PIL import Image
from os import listdir
from os.path import isfile, join
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

dir = '/content/train_images'
extension='jpg'

paths = []
for f in listdir(dir):
    if isfile(join(dir,f)):
        path = join(dir,f)
        number = f.split('_')[1]
        number = int(number.split('.')[0])
        paths.append((number, path)) 
paths.sort(key= lambda file: file[0])

imgs = []
for _, path in paths:
    image = Image.open(path).convert("RGB")
    image=np.asarray(image)
    imgs.append(image)

X = np.array(imgs)
X = X/127.5 -1.
y = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train_labels.csv')['Expected']

classes = np.unique(y)
n_classes = len(np.unique(y))
count = 0
labels ={}
for l in classes:
    labels[l] = count
    count+=1
y = np.array(y.replace(labels).tolist())
y = to_categorical(y,n_classes)

#Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40) 

del X
del y
del imgs

N, width, height, depth = X_train.shape
print(f'Cantidad de clases a predecir: {n_classes}')
print(f'Clases: {classes}')
print(f'Train shape:{X_train.shape}')
print(f'Test shape:{X_test.shape}')

Using TensorFlow backend.


Cantidad de clases a predecir: 10
Clases: ['altar' 'apse' 'bell_tower' 'column' 'dome_inner' 'dome_outer'
 'flying_buttress' 'gargoyle' 'stained_glass' 'vault']
Train shape:(7369, 128, 128, 3)
Test shape:(1843, 128, 128, 3)


Primero antes de resolver el problema, hay que decidir que arquitectura utilizar para el problema de clasificación de imagenes. Entre las opciones más conocidas se encuentran: VGG16, VGG19, Resnet, entre otras arquitecturas.

Una de las desventajas de las arquitecturas VGGXX es la cantidad de parametros a entrenar en el modelo en contraste a la profundidad de estos, en cambio las arquitecturas Resnet disminuye la cantidad de parámetros debido a la utilización de skip-connections, esto hace que estos modelos tengan una mayor profundidad. Debido a estas consideraciones se decidio utilizar una arquitectura resnet para abordar este problema.

Puesto que se tienen una cantidad acotada de datos (aprox ~ 10.000 imagenes), se decide utilizar la técnica de transfer learning de un modelo resnet.

# Resnet 50

In [5]:
import tensorflow as tf
from keras import Sequential
from keras.applications import ResNet152V2, ResNet50
from keras.optimizers import Adam, SGD, Adagrad
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten, UpSampling2D, GlobalAveragePooling2D, Dropout
from keras.models import Model

input_shape = (width, height, depth)
#newInput = Input(shape=(width, height, depth))
resnet_model = ResNet50(include_top=False,
                     weights="imagenet",
                     input_shape=input_shape)

model = Sequential()
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(n_classes, activation='softmax'))

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 8s 0us/step


En primera instancia se utilizo un modelo preetrenado Resnet con una profundidad de 50 capas, en este modelo se quitaron las capas densas del final del modelo, para agregar las siguientes capas:

1.   Average Pooling
2.   Dense (250 neuronas) con función de activación relu
3.   Dropout 
4.   Batch Normalizartion
5.   Softmax

Para un mejor desempeño de la red tambien se dejaron como entrenables las capas de batch que se encuentran entre las capas convolucionales del modelo.



## Resnet 152

In [0]:
import tensorflow as tf
from keras import Sequential
from keras.applications import ResNet152V2, ResNet50
from keras.optimizers import Adam, SGD, Adagrad
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten, UpSampling2D, GlobalAveragePooling2D, Dropout
from keras.models import Model

input_shape = (width, height, depth)
#newInput = Input(shape=(width, height, depth))
resnet_model = ResNet152V2(include_top=False,
                     weights="imagenet",
                     input_shape=input_shape)

model2 = Sequential()
model2.add(resnet_model)
model2.add(GlobalAveragePooling2D())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(.25))
#model2.add(BatchNormalization())
model2.add(Dense(n_classes, activation='softmax'))

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

En segunda instancia se hacen los mismos procedimientos pero en este caso se busca la mejora de resultados mediante el aumento de la profundidad del modelo.

## Augmented Data


En un principio se entrenaron los dos modelos anterior directamente  con los datos de entrenamiento, sin embargo, se observaba overfitting, es por esto que se busca mejorar este aspecto mediante la técnica Data Augmentation, la cual consiste en aplicar un conjunto de operaciones a las imagenes del conjunto de entrenamiento para que la red reciba más datos de entrenamiento pero que ahora en teoria para la imagen no son las mismas imagenes.

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True,
    vertical_flip=True
)
test_datagen = ImageDataGenerator()
train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train,
                                     y_train,
                                     batch_size=32
                                    )
test_generator = test_datagen.flow(X_test,
                                   y_test,
                                   batch_size=32
                                   )

Para para el entrenamiento se propone un scheduler para el descenso de la tasa de aprendizaje, epecificamente se propone un decaimiento de la tasa de apredizaje por pasos, es decir, cada cierta cantidad de epocas la tasa de apredizaje decae en un porcentaje. En especifico, en este problema se entreno con una tasa de 0.01, disminuyendola a la mitad cada 5 epocas. 

Se escogio una cantidad pequeña de epocas para el decaimiento debido a la utilización de un modelo preentrenado, por esto el modelo no debe diversificar mucho en el espacio de soluciones.

In [10]:
import math
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau


def step_decay(epoch):
   initial_lrate = 0.01
   drop = 0.5
   epochs_drop = 5
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate

checkpoint = ModelCheckpoint(filepath='final_model.h5', 
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)
lr_scheduler = LearningRateScheduler(step_decay)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
callbacks = [checkpoint, lr_reducer, lr_scheduler]


optimizer = Adam()
model2.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model2.summary()


model2.fit(train_generator,
                steps_per_epoch=len(X_train)//32,
                epochs=50,
                validation_data=test_generator,
                callbacks=callbacks
               )

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Model)          (None, 4, 4, 2048)        58331648  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 58,858,762
Trainable params: 670,858
Non-trainable params: 58,187,904
_________________________________________________________________
Epoch 1/50
230/230 [==============================] - 97s 420ms/step - loss: 0.8194 - accuracy: 0.7688

A partir de todo esto se escoge el modelo o movimiento dentro del espacio de soluciones que tiene mejor accuracy en el conjunto de test en las 100 epocas de entrenamiento, para evitar el overfitting. Con esto se llego a un accuracy cercano 96% en el conjunto de test.

In [1]:
from keras.models import load_model
resnetModel = load_model('/content/final_model.h5')

Using TensorFlow backend.


In [15]:
dir_test = '/content/test_images'
extension='jpg'

paths_test = []
# print(listdir(dir_test))
for f in listdir(dir_test):
    if isfile(join(dir_test,f)):
        path = join(dir_test,f)
        number = f.split('_')[1]
        number = int(number.split('.')[0])
        paths_test.append((number, path)) 
paths_test.sort(key= lambda file: file[0])

imgs_test = []
for _, path in paths_test:
    image_to_predict = Image.open(path).convert("RGB")
    image_to_predict=np.asarray(image_to_predict)
    imgs_test.append(image_to_predict)

X_kaggle= np.array(imgs_test)/127.5 - 1
X_kaggle.shape

(1023, 128, 128, 3)

In [0]:
y_kaggle = resnetModel.predict(X_kaggle)
y_kaggle = y_kaggle.argmax(axis=-1)

In [0]:
inv_labels = {v: k for k, v in labels.items()}

ids = [f'test_{i[0]}' for i in paths_test]

data_kaggle = pd.DataFrame({'Id': ids , 'Expected':y_kaggle})
data_kaggle['Expected'] = data_kaggle['Expected'].replace(inv_labels)
data_kaggle.to_csv('9no_intento.csv', index=False)